In [ ]:
# from lists_and_dicts import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
import plotly.express as px

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
plt.figure()
plt.rcParams.update({'font.size': 22}) # must set at top

# Contents
1. Factory Productivity

   1.1 raw material consumption by factory, month (value & qty, from stock_transactions.csv)

   1.2 finished goods production by factory, month (value & qty, from stock_transactions.csv)

   1.3 factory recoveries by  factory list, month (from financials)

2. Oil Supply

   2.1 tons received per month

   2.2 price per ton

3. Trading Performance

   3.1: sales revenue by cc list

   3.2: sales quantity by alt_product_group

           - tuffa tanks

           - GNS oil

           - dazzle bleach

   3.3: trading profit by cc list

   3.4: trading margins by cc list

4. Factory Performance

# 1. Factory Productivity
what does the data say about factory productivity? A few notes on the data used in this section. We're looking at stock transactions, specifically at the factory warehouses. In general there are three types of stock transactions we see in factory warehouses:
- type 0: finished goods are credited, associated with production
- type 1: raw materials are debited, associated with production
- type 4: goods are transferred between warehouses.

In general, raw materials are received into stock at WH06. Oil is the exception, where raw materials are also received at the factory warehouse WH40. Both raw material receipts and finished goods updates are type 0 stock transactions. In the oil factory case we need to differentiate on a third index field. transaction_type=0, warehouse=40, transaction_source=0

## 1.1 productivity - raw material consumption. (transaction type 1, factory warehouse, factory cc)

In [ ]:
stock = pd.read_csv('data/stock_transactions.csv', index_col=[0,1,3], low_memory=False)
new = stock['alt_product_group & cc'].str.split(n=0, expand=True)
stock['cc'] = new[0]
stock['alternate_product_group'] = new[1]
stock['value'] = stock['unit_cost'] * stock['quantity']
stock = stock.drop(columns=['alt_product_group & cc'])

# WK-ISSUE: It looks like NOT ALL 'alt_product_group & cc' comply to the format of 'cc' 'apg', some of them will have
#   the cost center number missing

# answer: yes you're right, and while it looks like this is a problem, in fact i think actually a good thing. it tells us something. the transactions that don't comply are not type 1 or type 0. I haven't looked at it super closely but they mainly look like intra-transfers. we're not interested in those.

num_invalid_apg = pd.isnull(stock['alternate_product_group']).sum()
print(f'You have {num_invalid_apg} records after the preprocessing where you find invalid "alternate_product_group"')
num_invalid_cc = pd.isnull(pd.to_numeric(stock['cc'], errors='coerce')).sum()
print(f'You have {num_invalid_cc} records after the preprocessing where you find invalid "cc"')

In [ ]:
# WK-ADVICE: From what I read from your the above, this is what I would do
def summarise_stock(trans_type, warehouse, cost_center_id):
	df = stock.xs((trans_type, warehouse))
	df = df.loc[df['cc'] == cost_center_id].copy()
	summary = df.pivot_table(index='month/year', values='value', aggfunc=np.sum)
	return summary


# transaction type 1
chemical_sum = summarise_stock(1, 48, '50')
oil_sum = summarise_stock(1, 40, '55')
plastics_sum = summarise_stock(1, 60, '60')
paper_sum = summarise_stock(1, 41, '80')
tuffa_sum = summarise_stock(1, 51, '85')

raw_mats_one = pd.DataFrame()
raw_mats_one = pd.concat([raw_mats_one, chemical_sum], axis=1).rename(columns={'value': 'chemical'})
raw_mats_one = pd.concat([raw_mats_one, oil_sum], axis=1).rename(columns={'value': 'oil'})
raw_mats_one = pd.concat([raw_mats_one, plastics_sum], axis=1).rename(columns={'value': 'plastics'})
raw_mats_one = pd.concat([raw_mats_one, paper_sum], axis=1).rename(columns={'value': 'paper'})
raw_mats_one = pd.concat([raw_mats_one, tuffa_sum], axis=1).rename(columns={'value': 'tuffa'})

In [ ]:
# what you've written is definitely better than mine, but isn't there not an easier way? when we do df.loc() on a dataframe with a multi index we use a tuple. Why can't we have a tuple of lists or a tuple of tuples? that would be very concise code. it might look something like this. NB: this doesn't work. it generates an error. this annoys me as i know it can be done. see these examples. they are using lists in their df.loc[()] calls. Can't figure out what i'm doing wrong:
# https://towardsdatascience.com/how-to-use-multiindex-in-pandas-to-level-up-your-analysis-aeac7f451fce
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d

# raw_materials = stock.loc[(1, FACTORY_COST_CENTERS_LIST)]

In [ ]:
raw_mats_one.head(20)

In [ ]:
ax = raw_mats_one.plot(kind='bar', stacked=True, figsize=(20, 12), fontsize='16', title='Factory Type 1', xlabel='Months', ylabel='Kina')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
# there are a couple of odd values but in general it passes the common sense test. the code is too complicated.
# the spike in chemical in april 20 is an error in the data

## raw material consumption (quantity)

In [ ]:
# transaction type 0
chemical_sum = summarise_stock(0, 48, '50')
oil_sum = summarise_stock(0, 40, '55')
plastics_sum = summarise_stock(0, 60, '60')
paper_sum = summarise_stock(0, 41, '80')
tuffa_sum = summarise_stock(0, 51, '85')

raw_mats_zero = pd.DataFrame()
raw_mats_zero = pd.concat([raw_mats_zero, chemical_sum], axis=1).rename(columns={'value': 'chemical'})
raw_mats_zero = pd.concat([raw_mats_zero, oil_sum], axis=1).rename(columns={'value': 'oil'})
raw_mats_zero = pd.concat([raw_mats_zero, plastics_sum], axis=1).rename(columns={'value': 'plastics'})
raw_mats_zero = pd.concat([raw_mats_zero, paper_sum], axis=1).rename(columns={'value': 'paper'})
raw_mats_zero = pd.concat([raw_mats_zero, tuffa_sum], axis=1).rename(columns={'value': 'tuffa'})

In [ ]:
raw_mats_zero.head(20)

In [ ]:
ax = raw_mats_zero.plot(kind='bar', stacked=True, figsize=(20, 12), fontsize='16', title='Factory Type 0', xlabel='Months', ylabel='Kina')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
# this looks ok. there are a couple of odd values but in general it passes the common sense test.the code is too complicated.
# the spike in chemical in april 20 is an error in the data
# there is double count going on with oil, which overstates the amount of oil for transaction type 0.

## Chemical Factory Raw Materials Quantity

In [ ]:
chemical_raw_materials = stock.loc[(1, 48, 'B')]  # need to only look at transaction type 1 from the factory warehouses. how can i pass a list of warehouses instead of one? if i can do that, then i can achieve the above with much shorter code
chemical_raw_materials.head()

In [ ]:
chemical_raw_materials = chemical_raw_materials.pivot_table(values='value', index='month/year', columns='warehouse', aggfunc=np.sum)
chemical_raw_materials.head(20)

In [ ]:
ax = chemical_raw_materials.plot(kind='bar', stacked=True, xlabel='Month', ylabel='Kina', title='Chemical Factory Type 1', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
# if we take out the weird number in april 20 which we know is an error, the data looks reasonable.

In [ ]:
chemical_factory_finished_goods = stock.loc[(0, 48, 'A')]
chemical_factory_finished_goods = chemical_factory_finished_goods.pivot_table(values='quantity', index='month/year', columns='warehouse', aggfunc=np.sum)
chemical_factory_finished_goods.head(20)

In [ ]:
ax = chemical_factory_finished_goods.plot(kind='bar', stacked=True, xlabel='Month', ylabel='Quantity', title='Chemical Factory Type 0', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

In [ ]:
chemical_factory_recoveries = chemical_factory_finished_goods + chemical_raw_materials
chemical_factory_recoveries.head(20)

In [ ]:
ax = chemical_factory_recoveries.plot(kind='bar', stacked=True, xlabel='Month', ylabel='Kina', title='Chemical Recoveries', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

## 1.2 oil output

In [ ]:
oil_finished_goods = stock.loc[(0, 40, 'A')]  # as above
oil_finished_goods = oil_finished_goods.pivot_table(values='quantity', index='month/year', columns='warehouse', aggfunc=np.sum)
oil_finished_goods.head(20)

In [ ]:
ax = oil_finished_goods.plot(kind='bar', stacked=True, title='Oil Factory Finished Goods Output (volume)', xlabel='Months', ylabel='Packs', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
# there is a problem with oil. Its showing both receipts and production finished goods, because both are transaction type 1,

In [ ]:
# for finished goods
chemical_sum = summarise_stock(0, 48, '50')
oil_sum = summarise_stock(0, 40, '55')
plastics_sum = summarise_stock(0, 60, '60')
paper_sum = summarise_stock(0, 41, '80')
tuffa_sum = summarise_stock(0, 51, '85')

finished_goods = pd.DataFrame()
finished_goods = pd.concat([finished_goods, chemical_sum], axis=1).rename(columns={'value': 'chemical'})
finished_goods = pd.concat([finished_goods, oil_sum], axis=1).rename(columns={'value': 'oil'})
finished_goods = pd.concat([finished_goods, plastics_sum], axis=1).rename(columns={'value': 'plastics'})
finished_goods = pd.concat([finished_goods, paper_sum], axis=1).rename(columns={'value': 'paper'})
finished_goods = pd.concat([finished_goods, tuffa_sum], axis=1).rename(columns={'value': 'tuffa'})

finished_goods.head(20)

In [ ]:
ax = finished_goods.plot(kind='bar', stacked=True, title='Finished Goods Output (value)', xlabel='Months', ylabel='Kina', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
# this looks pretty sensible, just need to invert it (multiply by -1)

## 1.1.3: Factory Recoveries

In [ ]:
financials = pd.read_excel('data/cc99_historic.xlsx', index_col='index')
# financials.head(150)
manufacturing_recoveries = financials.loc['Manufacturing Recoveries']
manufacturing_recoveries = manufacturing_recoveries.drop(labels=['accounting_code', 'cc', 'category', 'Group', 'description', 'alt_product_group'])
# manufacturing_recoveries.head(20)

In [ ]:
ax = manufacturing_recoveries.plot(kind='line', title='Factory Recoveries', rot=90, xlabel='Months', ylabel='Kina', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

## 1.1.4: Oil supply

## 1.1.5: Oil Price
RBD Palm Olein FOB Malaysia market price since January 2020

oil price for stock received from NBPOL

In [ ]:
oil_price = stock.loc[0, 40, '0']
oil_price = oil_price.pivot_table(values='unit_cost', columns='stock_code', index='month/year', aggfunc=np.mean)
oil_price = oil_price.drop('85-50-0010', axis=1)
oil_price.head(20)

In [ ]:
oil_price.plot(kind='line', title='Oil Price', rot=90, xlabel='Months', ylabel='Kina per kg', figsize=(20, 12), fontsize='16')

# 2. Trading Performance 2020 - 2021
## 2.1 Sales Revenue

In [ ]:
sales = pd.read_csv('data/sales_statistics.csv', low_memory=False)
new = sales['cc_and_alt_product_group'].str.split(n=0, expand=True)
sales['cc'] = new[0]
sales['alternate_product_group'] = new[1]
sales['cost_value'] = sales['cost_price_per_uom'] * sales['quantity']
sales['sales_revenue'] = sales['price_per_uom'] * sales['quantity']
sales['trading_profit'] = sales['sales_revenue'] - sales['cost_value']
sales['trading_margin'] = (sales['trading_profit'] / sales['sales_revenue']) * 100  # need to change the display format
sales = sales.drop(columns=['cc_and_alt_product_group'])
sales.head()

In [ ]:
sales_multi = sales.set_index(['order_type', 'cc', 'alternate_product_group'])
sales_multi.head()

In [ ]:
# going to have the same problems. need to be able to pass lists or dicts to loc()
lae_retail_sales = sales_multi.loc[(1, '14')]
# retail_sales.head()
lae_retail_sales = lae_retail_sales.pivot_table(values='sales_revenue', index='month/year', columns='alternate_product_group', aggfunc=np.sum)
lae_retail_sales = lae_retail_sales.drop(['ACBBOT', 'ACHCLE', 'ACKCOM', 'ACRSUN', 'ACUOIL', 'AIJSAF', 'AIPROT', 'ACMMAN'], axis=1)
lae_retail_sales.head(20)

In [ ]:
ax = lae_retail_sales.plot(kind='bar', stacked=True, title='lae retail sales revenue', xlabel='Months', ylabel='Kina', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
# need to group the APGs. Its too noisy right now. i think we should reduce to ACTDAZ, ACJDOM, and OTHER. Need to alias the APGs using a dict

## 2.2: Trading Profit

In [ ]:
lae_commercial_trading_profit = sales_multi.loc[(1, '11')]
lae_commercial_trading_profit = lae_commercial_trading_profit.pivot_table(values='trading_profit', index='month/year', columns='alternate_product_group', aggfunc=np.sum)
lae_commercial_trading_profit = lae_commercial_trading_profit.drop(['ACTDAZ', 'ACJDOM', 'ACTDHR', 'ACTDLP', 'RAWMAT', 'ACMCCR', 'AIXFRU', 'ACMPES', 'ACSONN', 'AICSPA', 'AIOSEH'], axis=1)
lae_commercial_trading_profit.head(20)

In [ ]:
ax = lae_commercial_trading_profit.plot(kind='bar', stacked=True, title='Lae Commercial Trading Profit', xlabel='Months', ylabel='Kina', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
# need to group these. Its too noisy right now. alias the APGs using SHORT_COMMERCIAL_APG_DICT. Can't make this work

In [ ]:
# commercial_trading_profit.to_csv('commercial_trading_profit.csv')
# question: how can i substitute the alt_product_group code for the name that is in the dictionary?
# question: need to find a way to aggregate multiple cost centres into commercial: process chemicals(22 & 44), plastics, etc

In [ ]:
sum_column = lae_commercial_trading_profit['ACBBOT'] + lae_commercial_trading_profit['ACCFOO'] +\
			 lae_commercial_trading_profit['ACHCLE'] + lae_commercial_trading_profit['ACIKAT'] +\
			 lae_commercial_trading_profit['ACSHYD'] + lae_commercial_trading_profit['ACRSUN'] +\
			 lae_commercial_trading_profit['ACSCAU']

# + lae_commercial_trading_profit['ACLCAT'] + lae_commercial_trading_profit['ACSENV'] + lae_commercial_trading_profit['ACSOOO'] + lae_commercial_trading_profit['AIJSAF'] + lae_commercial_trading_profit['AIPROT'] + lae_commercial_trading_profit['AIXFRU'] + lae_commercial_trading_profit['ACSALU']

# question: there is something wrong with this. why is it only showing figures up to september 20? the 'OTHER' column shoudl look like the one in commercial_trading_profit.csv

In [ ]:
sum_column.head(20)

In [ ]:
lae_commercial_trading_profit = lae_commercial_trading_profit.drop(['ACBBOT', 'ACCFOO', 'ACHCLE', 'ACIKAT', 'ACLCAT', 'ACRSUN', 'ACSALU', 'ACSCAU', 'ACSENV', 'ACSHYD', 'ACSOOO', 'AIJSAF', 'AIPROT'], axis=1)

In [ ]:
lae_commercial_trading_profit['OTHER'] = sum_column
lae_commercial_trading_profit.head(20)

In [ ]:
ax = lae_commercial_trading_profit.plot(kind='bar', stacked=True, title='Lae Commercial Trading Profit', xlabel='Months', ylabel='Kina', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

In [ ]:
sales.columns.tolist()

# 3.1 Sales Volumes of Key Product Categories
1. GNS cooking oil
2. dazzle bleach
3. tuffa tanks

In [ ]:
multi = sales.set_index(['order_type', 'cc', 'alternate_product_group'])
multi.head()

### Lae Oil Sales Quantity

In [ ]:
lae_oil_sales_quantity = multi.loc[(1, '19')]
lae_oil_sales_quantity = lae_oil_sales_quantity.pivot_table(values='quantity', index='month/year', columns='alternate_product_group', aggfunc=np.sum)
lae_oil_sales_quantity = lae_oil_sales_quantity.drop(['ACBBOT', 'ACBBOT', 'ACMMAN', 'ACRSUN', 'ACTDAZ', 'AIPROT', 'AIXFRU'], axis=1)  # drop erroneous data
lae_oil_sales_quantity.head(20)

In [ ]:
ax = lae_oil_sales_quantity.plot(kind='bar', stacked=True, title='Lae Oil Sales Quantity', xlabel='Months', ylabel='Quantity (Units)', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

### Lae Tuffa Sales Quantity

In [ ]:
tuffa_sales_quantity = multi.loc[(1, '29')]
tuffa_sales_quantity = tuffa_sales_quantity.pivot_table(values='quantity', index='month/year', columns='alternate_product_group', aggfunc=np.sum)
tuffa_sales_quantity = tuffa_sales_quantity.drop(['ACBBOT', 'ACJDOM', 'ACTDAZ', 'ACTDHR', 'ACTDLP', 'AIXFRU', 'RAWMAT'], axis=1)  # drop erroneous data
# tuffa_sales_quantity.head(20)

In [ ]:
ax = tuffa_sales_quantity.plot(kind='bar', stacked=True, title='Lae Tuffa Sales Quantity', xlabel='Months', ylabel='Quantity (Units)', figsize=(20, 12), fontsize='16')
ax.get_yaxis().set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))

In [ ]:
lae_oil_sales_quantity = multi.loc[(1, '19')]
lae_oil_sales_quantity = lae_oil_sales_quantity.pivot_table(values='quantity', index='month/year', columns='alternate_product_group', aggfunc=np.sum)
lae_oil_sales_quantity = lae_oil_sales_quantity.drop(['ACBBOT', 'ACMMAN', 'ACRSUN', 'ACTDAZ', 'AIPROT', 'AIXFRU'], axis=1)  # drop erroneous data
lae_oil_sales_quantity.head()